In [ ]:
import tensorflow as tf
import pandas as pd
from google.colab import files
import io
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.preprocessing import text
from tensorflow.keras.preprocessing.text import Tokenizer
from nltk.stem import PorterStemmer
import numpy as np
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding, LSTM, GRU, Bidirectional, SpatialDropout1D
from tensorflow.keras.preprocessing import text
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import TensorBoard
from nltk.tokenize import word_tokenize
import os
import pandas as pd
import string
from bs4 import BeautifulSoup
import re
import random
from tensorflow.keras.layers import Dropout
import os
from tensorflow.keras import datasets, layers, models
from tensorflow import keras
import datetime
import h5py
import pickle
from tensorflow.keras.callbacks import *
%load_ext tensorboard

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

uploaded = files.upload()

df = pd.read_csv(io.StringIO(uploaded["training.1600000.processed.noemoticon.csv"].decode("Latin-1")))


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
df = pd.read_csv(io.StringIO(uploaded["training.1600000.processed.noemoticon.csv"].decode("Latin-1")))
df['target'] = df["target"].replace(4, 1) # replace labels of 4 with 1, easier for the computer to compute
df.drop(["id","flag","user"],axis=1,inplace=True) # drop unneeded columns

In [ ]:
# find the amount of distinct words found in the dataset
full_text = []
for row in df["text"]:
    row = row.split()
    full_text += row

distinct_words = list(dict.fromkeys(full_text))
print("Found " + str(len(distinct_words)) + " unique words")

Found 1350598 unique words


In [ ]:
X = df["text"]
y = df["target"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

tokenizer = Tokenizer(num_words=len(distinct_words)) 
tokenizer.fit_on_texts(X) 
word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train) 
X_test = tokenizer.texts_to_sequences(X_test)
print("complete")

complete


In [ ]:
X_train = np.array(X_train)
X_test = np.array(X_test)
y_train = np.array(y_train)
y_test = np.array(y_test)

X_train = sequence.pad_sequences(X_train, maxlen=280)
X_test = sequence.pad_sequences(X_test, maxlen=280)
X_test[0].shape
print("complete")

complete


In [ ]:
with open('/content/gdrive/My Drive/Colab Notebooks/CheckPoints/LSTM-TwoLayer/tokenizerBruhhhh.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:


max_features = len(distinct_words) + 1 # unique vocab
maxlen = 280 # the number of words per data point
embedding_size = 264 # the dimensions that each word is converted to

logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

filepath="/content/gdrive/My Drive/Colab Notebooks/CheckPoints/LSTM-TwoLayer/theepochs:{epoch:03d}-val_acc:{val_accuracy:.3f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor="val_accuracy", verbose=1,
                             save_best_only=True, mode='max')

callbacks_list = [checkpoint]
callbacks_list.append(tensorboard_callback)

completed1


In [ ]:
model = Sequential()

model.add(Embedding(max_features, embedding_size, input_length=maxlen) )

model.add(SpatialDropout1D(0.3))

model.add( GRU(264, dropout=0.3, recurrent_dropout=0.3, return_sequences=True) )

model.add( GRU(264, dropout=0.3, recurrent_dropout=0.3) ) 

model.add( Dense(1, activation='sigmoid') )

model.compile(loss='binary_crossentropy',
              optimizer="adam",
              metrics=['accuracy'] )

fitModel = model.fit(X_train, y_train, epochs = 7, batch_size = 324, validation_data=(X_test, y_test), 
                     verbose=1, callbacks=callbacks_list) 

print( model.summary() )

Epoch 1/7
3704/3704 [==============================] - ETA: 0s - loss: 0.4150 - accuracy: 0.8090
Epoch 00001: val_accuracy improved from -inf to 0.82786, saving model to /content/gdrive/My Drive/Colab Notebooks/CheckPoints/LSTM-TwoLayer/theepochs:001-val_acc:0.828.hdf5
3704/3704 [==============================] - 13175s 4s/step - loss: 0.4150 - accuracy: 0.8090 - val_loss: 0.3814 - val_accuracy: 0.8279
Epoch 2/7
3704/3704 [==============================] - ETA: 0s - loss: 0.2928 - accuracy: 0.8762
Epoch 00002: val_accuracy did not improve from 0.82786
3704/3704 [==============================] - 12721s 3s/step - loss: 0.2928 - accuracy: 0.8762 - val_loss: 0.4349 - val_accuracy: 0.8082
Epoch 3/7
 458/3704 [==>...........................] - ETA: 2:50:37 - loss: 0.2207 - accuracy: 0.9092